### Песочница
    > полезные ссылки: 
        https://habr.com/ru/post/418281/ - "Как решить 90% задач NLP"
        https://webdevblog.ru/gensim-rukovodstvo-dlya-nachinajushhih/ - "Gensim. Руководство для начинающих"
        https://www.ibm.com/developerworks/ru/library/os-pythonnltk/
        http://www.nltk.org/
        https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
        https://github.com/hhru/api

In [2]:
import sqlite3
from bs4 import BeautifulSoup

In [3]:
# обработка базы данных

original_db = 'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\sberbank_hh.db'
new_db = 'C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\updated_vacancies.db'


conn = sqlite3.connect(new_db)
cursor = conn.cursor()

#cursor.execute('SELECT * from vacancy')
cursor.execute('SELECT * from vacancies')
results = cursor.fetchall()

conn.close()

In [4]:
len(results)

1816

In [140]:
results[110]

('https://hh.ru/vacancy/29695558',
 'Старший клиентский менеджер (крупный и средний бизнес)',
 'Москва',
 '2019-01-16',
 'Функции:формирование потребности и развитие долгосрочных отношений с клиентами крупного-среднего бизнеса;активные продажи всех банковских продуктов действующим и новым клиентам крупного-среднего бизнеса;формирование и координация работы клиентско-продуктовой команды в рамках реализации и сопровождения сделок с клиентами крупного-среднего бизнеса;сбор документов, предварительный анализ кредитоспособности, сопровождение и согласование кредитных сделок;бизнес-планирование, формирование отчетности и клиентская аналитика;участвует в мониторинге и работе с проблемной задолженностью (до 90 дней). Требования:высшее образование - финансы / банки, экономика / бухгалтерия;опыт работы в должности Клиентского менеджер (крупный, средний бизнес) от 1-го года;опыт привлечения клиентов;знание действующего законодательства РФ в части вопросов, связанных с кредитованием корпоративных 

### Создание новой базы данных
    > удаление дубликатов
    > приведение к более удобочитаемому виду

In [6]:
conn = sqlite3.connect('updated_vacancies.db')
cursor = conn.cursor()

In [181]:
def get_descr(raw_html):
    soup = BeautifulSoup(raw_html)
    return soup.find('div', class_='g-user-content').text.strip()

In [154]:
conn = sqlite3.connect('C:\\Users\\busya\\Desktop\\anaconda\\проекты\\курсовая\\updated_vacancies.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE vacancies (link text, name text, city text, release_date text, description text)''')

In [155]:
links = []
errors = []

for vacancy in results:
    
    if vacancy[1] in links:
        continue
    
    else:
        links.append(vacancy[1])
        vac_set = (
                    vacancy[1], # link
                    vacancy[2], # vacancy name
                    vacancy[3].split('<p>')[3][8:-4], # city name
                    vacancy[4][:10], # date
                    get_descr(vacancy[5]) # description
        )
        try:
            cursor.execute(f'''INSERT INTO vacancies VALUES {vac_set}''')
        except:
            errors.append(vacancy[1])
conn.commit()

In [166]:
errors # некорректное описание (использованы спецсимволы)

['https://hh.ru/vacancy/27408489', 'https://hh.ru/vacancy/29810176']

In [167]:
cursor.execute('SELECT * from vacancies')
updated_res = cursor.fetchall()
conn.close()

In [168]:
len(results) - len(updated_res) # удалено вакансий

444

### Попытка решить что-то вручную

In [42]:
def check_it(text):
    ans = False
    possible_definitions = ['Ты подходишь нам, если ты', 'Требования', 'Навыки', 'Технические компетенции', 'Если есть',
                           'ТРЕБОВАНИЯ К КАНДИДАТУ', 'Необходимые навыки и технологии', 'Что мы ждем от кандидата',
                           'Мы ожидаем от Вас', 'Мы ждем, что ты', 'Профессиональные компетенции', 'Ты подходишь нам, если',
                           'Профессиональные навыки', 'Мы ждём от тебя', 'Мы ожидаем, что ты', 'Наши ожидания',
                           'Что мы хотим в вас видеть?', 'От Вас нужно', 'Стэк',
                           'Мы определили следующие обязательные требования к кандидату', 'Требование', 'Общие требования',
                           'Требуемые навыки', 'Профессиональные требования', 'Обязательно', 'От вас нужно', 'Чего ждем от кандидата',
                           'Что ждем', 'Опыт работы с технологиями', 'Мы ожидаем', 'ТЕХНИЧЕСКИЕ КОМПЕТЕНЦИИ', 'ТРЕБОВАНИЯ',
                           'Знание технологий', 'Опыт:', 'Обязательные требования', 'Мы ждем что вы умеете/обладаете']
    
    for def_ in possible_definitions:
        if def_ in text:
            ans = True
            break
    return ans

In [44]:
simple_texts = 0
other = []
for i in range(len(results)):
    if check_it(results[i][4]):
        simple_texts += 1
    else:
        other.append(i)

In [45]:
simple_texts

1772

In [46]:
other[:10]

[99, 268, 273, 301, 391, 443, 578, 670, 745, 746]

In [ ]:
nontrivial = [194, 220]

### Обработка наименований вакансий

In [11]:
job_names = {} # словарь профессий с записью индексов повторяющихся позиций
for i in range(len(results)):
    name = results[i][1]
    if name in job_names:
        job_names[name].append(i)
    else:
        job_names[name] = [i]
        

In [23]:
list(job_names.keys())[:10]

['Менеджер по продажам продуктов эквайринга',
 'Консультант',
 'Инженер технической поддержки автоматизированных рабочих мест',
 'Frontend разработчик',
 'JavaScript разработчик',
 'Ведущий инженер',
 'Специалист по прямым продажам',
 'Специалист по кадрам',
 'Менеджер по продажам',
 'Директор филиала лизинговой компании']

In [136]:
job_names['Java-разработчик']

[115,
 227,
 335,
 386,
 671,
 1054,
 1055,
 1056,
 1057,
 1058,
 1059,
 1060,
 1061,
 1164,
 1221,
 1450]

In [21]:
job_lengthes = {}
for job in list(job_names.keys()):
    job_lengthes[job] = len(job_names[job])

In [37]:
sorted(list(job_lengthes.items()), key = lambda x: x[1], reverse=True)

[('Консультант', 300),
 ('Менеджер по продажам', 138),
 ('Специалист по прямым продажам', 94),
 ('Старший менеджер по обслуживанию', 57),
 ('Менеджер по работе с ключевыми клиентами', 33),
 ('Менеджер по обслуживанию', 29),
 ('Специалист по взысканию задолженности', 26),
 ('Клиентский менеджер Сбербанк Премьер', 23),
 ('Java-разработчик', 16),
 ('Менеджер по продажам продуктов эквайринга', 14),
 ('Клиентский менеджер', 14),
 ('Старший специалист по взысканию', 13),
 ('Системный аналитик', 12),
 ('Java разработчик', 12),
 ('Клиентский менеджер малого бизнеса', 9),
 ('Старший кредитный аналитик', 9),
 ('Менеджер по работе с ключевыми клиентами малого бизнеса', 9),
 ('Водитель-инкассатор', 9),
 ('Кредитный инспектор', 8),
 ('Кассир', 8),
 ('Юрисконсульт', 8),
 ('Главный инженер по разработке', 7),
 ('Старший юрисконсульт', 7),
 ('Клиентский менеджер отдела продаж лизинга крупному и среднему бизнесу', 7),
 ('Водитель', 7),
 ('Аналитик', 6),
 ('Старший специалист', 6),
 ('Сервис-менеджер', 